In [1]:
#import
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd
import regex as re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
import json

class HealthSpider(scrapy.Spider):
    name = 'health'
    start_urls = ['https://www.webteb.com/diseases']

    def parse(self, response):
        items = response.css('div.content.acc-content ul.list li')
        for item in items:
            disease_type = item.css('a::text').get().strip()
            disease_link = item.css('a::attr(href)').get()
            base_link = disease_link.split('/diseases')[0]
            full_link = response.urljoin(base_link + "/diseases/")
            yield scrapy.Request(url=full_link, callback=self.parse_disease_list, meta={'disease_type': disease_type, 'link': full_link})

    def parse_disease_list(self, response):
        disease_type = response.meta['disease_type']
        diseases = response.css('div.search-result.border div.content')
        for disease in diseases:
            disease_name = disease.css('a.bold span::text').get().strip()
            disease_link = disease.css('a.bold::attr(href)').get()
            full_disease_link = response.urljoin(disease_link)
            yield scrapy.Request(url=full_disease_link, callback=self.parse_disease_details, meta={'disease_type': disease_type, 'disease_name': disease_name})

    def parse_disease_details(self, response):
        disease_type = response.meta['disease_type']
        disease_name = response.meta['disease_name']
        
        # Extracting data for each section
        description = response.css('div#description').xpath('string()').get().strip()
        symptoms_title = response.css('div#symptoms > h2::text').get()
        symptoms = '\n'.join(response.css('div#symptoms ul li::text').getall())
        causes_and_risk_factors_title = response.css('div#causesandriskfactors > h2::text').get()
        causes_and_risk_factors = '\n'.join(response.css('div#causesandriskfactors ul li::text').getall())
        complications_title = response.css('div#complications > h2::text').get()
        complications = '\n'.join(response.css('div#complications ul li::text').getall())
        diagnosis_title = response.css('div#diagnosis > h2::text').get()
        diagnosis = '\n'.join(response.css('div#diagnosis ul li::text').getall())
        treatments_and_drugs_title = response.css('div#treatementsanddrugs > h2::text').get()
        treatments_and_drugs = '\n'.join(response.css('div#treatementsanddrugs p::text').getall())
        prevention_title = response.css('div#prevention > h2::text').get()
        prevention = '\n'.join(response.css('div#prevention ul li::text').getall())

        content = f"{disease_type}\n{disease_name}\n{description}\n\n{symptoms_title}:\n{symptoms}\n\n{causes_and_risk_factors_title}:\n{causes_and_risk_factors}\n\n{complications_title}:\n{complications}\n\n{diagnosis_title}:\n{diagnosis}\n\n{treatments_and_drugs_title}:\n{treatments_and_drugs}\n\n{prevention_title}:\n{prevention}"

        yield {
            'disease_type': disease_type,
            'disease_name': disease_name,
            'content': content
        }


# Create a process to run the spider
process = CrawlerProcess(settings={
    'FEEDS': {
        'disease_details2.json': {
            'format': 'json',
            'encoding': 'utf8',
            'overwrite': True,
        },
    },
})

# Run the spider
process.crawl(HealthSpider)
process.start()

# Load and display the data from the JSON file
with open('disease_details2.json', 'r', encoding='utf8') as f:
    disease_details = json.load(f)

# Display the extracted data
for item in disease_details:
    print(item['content'])


In [2]:
import pandas as pd
import json

# Load your JSON data with utf-8 encoding
with open('disease_details1.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Create a DataFrame
df = pd.DataFrame(data)

# Add an empty column named "score"
df = df.assign(score='')

# Display the first few rows
df.head()

,disease_type,disease_name,content,score
0,تسوس الأسنان,امراض الاغشية المخاطية للفم,تسوس الأسنان\nامراض الاغشية المخاطية للفم\nتغط...,
1,تسوس الأسنان,امراض التهاب اللب,تسوس الأسنان\nامراض التهاب اللب\nقد يكون الالت...,
2,تسوس الأسنان,عيوب خلقية لجوف الفم,تسوس الأسنان\nعيوب خلقية لجوف الفم\nيوجد للفم ...,
3,مرض السكري,حساسية الإنسولين,مرض السكري\nحساسية الإنسولين\nحتى نهاية سنوات ...,
4,تسوس الأسنان,ساركوما كابوزي,تسوس الأسنان\nساركوما كابوزي\nساركوما كابوزي ع...,


In [3]:
import pandas as pd
import json

# Define the score_text function
def score_text(text):
    # Define your scoring rules here
    score = 0
    
    # Keywords related to symptoms
    symptoms_keywords = ['ألم', 'عرض', 'علامة', 'ظهور', 'تشعر', 'تشعرين', 'يشعر', 'يعاني', 'يعانون', 'يعاني من', 'يعانون من', 'أعراض جانبية', 'تأثيرات جانبية', 'تغييرات في الجسم', 'وظائف غير طبيعية', 'مشاكل صحية', 'انزعاج', 'إزعاج', 'وجه', 'رأس', 'رقبة', 'صدر', 'ظهر', 'بطن', 'ذراع', 'ساق', 'مفصل', 'عضلة', 'جلد', 'شعر', 'أظافر', 'فم', 'لسان', 'أسنان', 'لثة', 'عيون', 'أذنين', 'أنف', 'شديد', 'حاد', 'خفيف', 'متوسط', 'مستمر', 'متقطع', 'مزمن', 'لا يطاق', 'غير محتمل', 'مُنهِك', 'مُؤلم', 'يُعيق', 'يُضعف', 'يُسبب إعاقة', 'حرقة', 'وخز', 'خدر', 'حكة', 'ألم غامض', 'ضعف', 'تشنج', 'غثيان', 'إقياء', 'إسهال', 'إمساك', 'سعال', 'ضيق تنفس', 'صعوبة في البلع', 'دوار', 'إغماء', 'نزيف', 'طفح جلدي', 'احمرار', 'تورم', 'جفاف', 'تقشر', 'تشقق', 'حكة', 'ألم عند اللمس', 'حساسية للمس', 'حساسية للحرارة', 'حساسية للبرودة', 'حساسية للضوء', 'حساسية للصوت', 'فقدان حاسة التذوق', 'فقدان حاسة الشم', 'قلق', 'اكتئاب', 'صداع', 'إرهاق', 'عصبية', 'توتر', 'اضطراب', 'مشاكل في النوم', 'ارتباك', 'تهيج', 'عدوانية', 'هلوسة', 'أوهام', 'فقدان التركيز', 'صعوبة في التفكير', 'صعوبة في اتخاذ القرارات', 'تغييرات في المزاج', 'سلوكيات غير طبيعية', 'ارتفاع درجة الحرارة', 'قشعريرة', 'حمى', 'قشعريرة', 'إفرازات مهبلية غير طبيعية', 'آلام الحوض', 'نزيف بين الدورات', 'نزيف بعد انقطاع الطمث', 'صعوبة التبول', 'إلحاح بولي', 'سلس البول', 'دم في البول', 'إفرازات قطنية بيضاء من المهبل', 'حرقان عند التبول', 'فرط التبول', 'تبول ليلي', 'ضعف الانتصاب', 'قلة الشهية']
    for keyword in symptoms_keywords:
        if keyword in text:
            score += 1
  
    # Keywords related to causes and risk factors
    causes_and_risk_factors_keywords = ['سبب', 'مسبب', 'عامل', 'يسبب', 'يؤدي إلى', 'يزيد من خطر', 'يقلل من خطر', 'مرض', 'عدوى', 'فيروس', 'بكتيريا', 'فطر', 'طفيلي', 'ورم', 'سرطان', 'خلل', 'اضطراب', 'حالة', 'وراثي', 'تاريخ عائلي', 'غذاء', 'تمارين', 'سمنة', 'تدخين', 'كحول', 'مخدرات', 'إجهاد', 'قلة نوم', 'تلوث', 'مواد كيميائية', 'إشعاع', 'لدغة', 'حشرة', 'إصابة', 'حادث', 'كسر', 'جرح', 'عمر', 'عوامل نفسية', 'ضغط عصبي', 'عوامل اجتماعية اقتصادية', 'فقر', 'انعدام الأمن الغذائي', 'إدمان الكحول والمخدرات', 'التعرض للعنف', 'التعرض للسموم', 'عوامل وراثية', 'طفرات جينية', 'عيوب خلقية', 'خلل التوازن الهرموني', 'أمراض المناعة الذاتية', 'استعداد وراثي', 'استهلاك الأدوية', 'التاريخ الطبي الشخصي', 'أمراض سابقة', 'الحمل', 'الرضاعة الطبيعية', 'سن اليأس', 'الجنس', 'العرق', 'النشاط البدني', 'تعرض أشعة الشمس']
    for keyword in causes_and_risk_factors_keywords:
        if keyword in text:
            score += 1
    
    # Keywords related to complications
    complications_keywords = ['مضاعفات', 'مضاعفة', 'يعقد', 'يؤدي إلى', 'يزيد من خطر', 'إعاقة', 'ضعف', 'فقدان وظيفة', 'اكتئاب', 'قلق', 'ألم مزمن', 'انتشار', 'وفاة', 'يهدد الحياة', 'تلف دائم', 'فشل عضوي', 'غيبوبة', 'عجز دائم', 'تشوهات', 'ندبات', 'التهاب مزمن', 'اضطرابات نفسية', 'اعتمادية على العلاج', 'انتكاس المرض', 'موت الأنسجة', 'نقص المناعة', 'سرطان ثانوي']
    for keyword in complications_keywords:
        if keyword in text:
            score += 1
    
    # Keywords related to diagnosis
    diagnosis_keywords = ['تشخيص', 'تشخيصي', 'يحدد', 'يكشف عن', 'يقيّم', 'تاريخ طبي', 'أعراض', 'فحص جسدي', 'فحص سريري', 'تحليل', 'فحص', 'تصوير', 'أشعة سينية', 'أشعة مقطعية', 'تصوير بالرنين المغناطيسي', 'تحليل دم', 'تحليل بول', 'خزعة', 'تنظير داخلي', 'تخطيط كهربية القلب', 'تخطيط كهربية الدماغ', 'قياس ضغط الدم', 'قياس السكر', 'اختبارات وظائف الكلى', 'اختبارات وظائف الكبد', 'اختبارات الحساسية', 'اختبارات الحمض النووي', 'فحص الحمض النووي', 'فحص جيني']
    for keyword in diagnosis_keywords:
        if keyword in text:
            score += 1
    
    # Keywords related to treatments and drugs
    treatments_and_drugs_keywords = ['علاج', 'علاج طبي', 'دواء', 'جرعة', 'جراحة', 'إشعاع', 'علاج طبيعي', 'علاج إشعاعي', 'علاج كيماوي', 'علاج بالهرمونات', 'علاج بالضوء', 'علاج سلوكي معرفي', 'علاج بالوخز بالإبر', 'علاج بالاسترخاء', 'علاج بالفن', 'علاج بالحمية الغذائية', 'علاج بديل', 'علاج تكميلي', 'جراحة من الحد الأدنى', 'جراحة تنظيرية', 'علاج بالخلايا الجذعية', 'علاج بالجينات', 'العلاج المناعي', 'المضادات الحيوية', 'مضادات الفيروسات', 'مضادات الفطريات', 'مضادات الطفيليات', 'مسكنات الألم', 'خافضات الحرارة', 'مضادات الالتهاب', 'مضادات الا']
    for keyword in treatments_and_drugs_keywords:
        if keyword in text:
            score += 1
    
    # Keywords related to prevention
    prevention_keywords = ['الوقاية', 'التوقي', 'الوقاية من', 'التوقي من', 'تجنب', 'تقليل', 'تحسين', 'الحد من', 'الوقاية الأولية', 'التوعية', 'التثقيف', 'التوجيه', 'النصائح', 'التوجيهات', 'التدابير', 'التدابير الوقائية', 'الإجراءات الوقائية', 'تغيير نمط الحياة', 'النظام الغذائي الصحي', 'ممارسة الرياضة', 'تجنب التدخين', 'التقليل من تناول الكحول', 'التقليل من تناول المخدرات', 'النوم الجيد', 'التقليل من التوتر', 'تجنب التعرض للعوامل الملوثة', 'التقليل من التعرض للإشعاع', 'استخدام وسائل الوقاية', 'التطعيم', 'التطهير', 'التعقيم', 'تنظيف اليدين', 'تنظيف البيئة', 'التغذية السليمة', 'العناية بالصحة النفسية', 'التوجيهات الطبية', 'الفحوصات الدورية', 'الكشف المبكر']
    for keyword in prevention_keywords:
        if keyword in text:
            score += 1
    
    return score

# Apply the score_text function to the 'content' column and update the 'score' column
df['score'] = df['content'].apply(score_text)

# Display the updated DataFrame
df.head()

,disease_type,disease_name,content,score
0,تسوس الأسنان,امراض الاغشية المخاطية للفم,تسوس الأسنان\nامراض الاغشية المخاطية للفم\nتغط...,37
1,تسوس الأسنان,امراض التهاب اللب,تسوس الأسنان\nامراض التهاب اللب\nقد يكون الالت...,37
2,تسوس الأسنان,عيوب خلقية لجوف الفم,تسوس الأسنان\nعيوب خلقية لجوف الفم\nيوجد للفم ...,42
3,مرض السكري,حساسية الإنسولين,مرض السكري\nحساسية الإنسولين\nحتى نهاية سنوات ...,20
4,تسوس الأسنان,ساركوما كابوزي,تسوس الأسنان\nساركوما كابوزي\nساركوما كابوزي ع...,25


In [4]:
def normalize_score(df, column_name):
    # Find the minimum and maximum scores
    min_score = df[column_name].min()
    max_score = df[column_name].max()
    
    # Normalize the scores between 0 and 10
    df['score'] = (df[column_name] - min_score) * 10 / (max_score - min_score)
    
    return df

# Example usage:
# Assuming df is your DataFrame and 'score' is the column containing scores
# Replace 'score' with the actual column name in your DataFrame
df = normalize_score(df, 'score')
df.head(40)

,disease_type,disease_name,content,score
0,تسوس الأسنان,امراض الاغشية المخاطية للفم,تسوس الأسنان\nامراض الاغشية المخاطية للفم\nتغط...,5.384615
1,تسوس الأسنان,امراض التهاب اللب,تسوس الأسنان\nامراض التهاب اللب\nقد يكون الالت...,5.384615
2,تسوس الأسنان,عيوب خلقية لجوف الفم,تسوس الأسنان\nعيوب خلقية لجوف الفم\nيوجد للفم ...,6.153846
3,مرض السكري,حساسية الإنسولين,مرض السكري\nحساسية الإنسولين\nحتى نهاية سنوات ...,2.769231
4,تسوس الأسنان,ساركوما كابوزي,تسوس الأسنان\nساركوما كابوزي\nساركوما كابوزي ع...,3.538462
5,تسوس الأسنان,جفاف الفم,تسوس الأسنان\nجفاف الفم\nيُساعد اللعاب على منع...,5.538462
6,تساقط الشعر,تساقط الشعر,تساقط الشعر\nتساقط الشعر\nأي شخص يُلاحظ أن شعر...,3.692308
7,تسوس الأسنان,رائحة الفم,تسوس الأسنان\nرائحة الفم\nيعاني العديد من الأش...,3.384615
8,الحمل المنتبذ,الولادة المبكرة,الحمل المنتبذ\nالولادة المبكرة\nيستمر الحمل ال...,4.461538
9,الحمل المنتبذ,الإجهاض,الحمل المنتبذ\nالإجهاض\nالإجهاض هو الفقد التلق...,7.076923


In [6]:
import re
import pandas as pd

# Define the basic_cleaning function
def basic_cleaning(text):
    text = str(text)  # Convert to string to handle non-string inputs
    text = text.lower()  # Lowercasing
    text = re.sub(r'{.*?}', '', text)   # Remove CSS code within curly braces
    # Removing all except alphanumeric characters, spaces, and specified characters
    text = re.sub(r'[^\w\s\-\+\*/!()\[\]{}<>]', '', text)
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove CSS
    text = re.sub(r'<style[^>]*>[\s\S]+?</style>', '', text)
    # Remove '\n' and replace with space
    text = text.replace('\n', ' ')
    # Remove non-Arabic characters and non-Arabic numbers
    text = re.sub(r'[^\u0621-\u064A0-9\s\-\+\*/!\[\]{}<>]', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Fill any NaN values with an empty string
df.fillna('', inplace=True)

In [7]:
def tokenize(text):
    return word_tokenize(text)

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

def stem_tokens(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in tokens]

def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in tokens]

def preprocess_text(df, text_column):
    df['cleaned_text'] = df[text_column].apply(basic_cleaning)
    df['tokens'] = df['cleaned_text'].apply(tokenize)
    df['tokens'] = df['tokens'].apply(remove_stopwords)
    df['stemmed_tokens'] = df['tokens'].apply(lambda x: ' '.join(stem_tokens(x)))
    df['lemmatized_tokens'] = df['tokens'].apply(lambda x: ' '.join(lemmatize_tokens(x)))
    return df

preprocessed_df = preprocess_text(df, 'content')
preprocessed_df.head()

,disease_type,disease_name,content,score,cleaned_text,tokens,stemmed_tokens,lemmatized_tokens
0,تسوس الأسنان,امراض الاغشية المخاطية للفم,تسوس الأسنان\nامراض الاغشية المخاطية للفم\nتغط...,5.384615,تسوس الأسنان امراض الاغشية المخاطية للفم تغطي ...,"[تسوس, الأسنان, امراض, الاغشية, المخاطية, للفم...",تسوس الأسنان امراض الاغشية المخاطية للفم تغطي ...,تسوس الأسنان امراض الاغشية المخاطية للفم تغطي ...
1,تسوس الأسنان,امراض التهاب اللب,تسوس الأسنان\nامراض التهاب اللب\nقد يكون الالت...,5.384615,تسوس الأسنان امراض التهاب اللب قد يكون الالتها...,"[تسوس, الأسنان, امراض, التهاب, اللب, قد, يكون,...",تسوس الأسنان امراض التهاب اللب قد يكون الالتها...,تسوس الأسنان امراض التهاب اللب قد يكون الالتها...
2,تسوس الأسنان,عيوب خلقية لجوف الفم,تسوس الأسنان\nعيوب خلقية لجوف الفم\nيوجد للفم ...,6.153846,تسوس الأسنان عيوب خلقية لجوف الفم يوجد للفم وظ...,"[تسوس, الأسنان, عيوب, خلقية, لجوف, الفم, يوجد,...",تسوس الأسنان عيوب خلقية لجوف الفم يوجد للفم وظ...,تسوس الأسنان عيوب خلقية لجوف الفم يوجد للفم وظ...
3,مرض السكري,حساسية الإنسولين,مرض السكري\nحساسية الإنسولين\nحتى نهاية سنوات ...,2.769231,مرض السكري حساسية الإنسولين حتى نهاية سنوات ال...,"[مرض, السكري, حساسية, الإنسولين, حتى, نهاية, س...",مرض السكري حساسية الإنسولين حتى نهاية سنوات ال...,مرض السكري حساسية الإنسولين حتى نهاية سنوات ال...
4,تسوس الأسنان,ساركوما كابوزي,تسوس الأسنان\nساركوما كابوزي\nساركوما كابوزي ع...,3.538462,تسوس الأسنان ساركوما كابوزي ساركوما كابوزي عبا...,"[تسوس, الأسنان, ساركوما, كابوزي, ساركوما, كابو...",تسوس الأسنان ساركوما كابوزي ساركوما كابوزي عبا...,تسوس الأسنان ساركوما كابوزي ساركوما كابوزي عبا...


In [8]:
#importing the libraries for encoding
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import numpy as np

In [ ]:
print(preprocessed_df['cleaned_text'][983])

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Bidirectional, GRU, LSTM, Dense

In [13]:
# Assuming you have already preprocessed your text data and embedded it with TF-IDF
# Adjust the following based on your actual data
max_sequence_length = 100  # Set an appropriate sequence length
num_features = 300  # Number of features after embedding

# Split data into features (X) and target (y)
X = df['cleaned_text']  # Assuming 'content' contains preprocessed text features
y = df['score']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a simple RNN model
model_rnn = build_rnn_model(input_shape=(max_sequence_length, num_features), num_classes=1)
model_rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model_rnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Create a bidirectional RNN model
model_bidirectional_rnn = build_bidirectional_rnn_model(input_shape=(max_sequence_length, num_features), num_classes=1)
model_bidirectional_rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model_bidirectional_rnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Create a GRU model
model_gru = build_gru_model(input_shape=(max_sequence_length, num_features), num_classes=1)
model_gru.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model_gru.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Create an LSTM model
model_lstm = build_lstm_model(input_shape=(max_sequence_length, num_features), num_classes=1)
model_lstm.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model_lstm.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

NameError: name 'build_rnn_model' is not defined

In [14]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['lemmatized_tokens'])
sequences = tokenizer.texts_to_sequences(df['lemmatized_tokens'])

# Pad the sequences
max_sequence_length = max([len(seq) for seq in sequences])
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Prepare the labels
y = df['score'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'Tokenizer' is not defined

In [15]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Assuming your DataFrame is named preprocessed_df
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_df['lemmatized_tokens'])
sequences = tokenizer.texts_to_sequences(preprocessed_df['lemmatized_tokens'])

# Pad the sequences
max_sequence_length = max([len(seq) for seq in sequences])
X = pad_sequences(sequences, maxlen=max_sequence_length)

# Prepare the labels
y = preprocessed_df['score'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
X_test

array([[    0,     0,     0, ..., 26314,   716, 11504],
       [    0,     0,     0, ...,   198,  1415,  2025],
       [    0,     0,     0, ...,   159,     1,  5806],
       ...,
       [    0,     0,     0, ...,   840,  2139,  3764],
       [    0,     0,     0, ...,     2,  1108,   314],
       [    0,     0,     0, ...,   481,     1,   408]])

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

def create_rnn_model(input_shape):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=input_shape))
    model.add(SimpleRNN(128))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

rnn_model = create_rnn_model(X_train.shape[1])
rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - loss: 10.8071 - mae: 2.7352 - val_loss: 1.5545 - val_mae: 0.9734
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - loss: 1.7344 - mae: 1.0124 - val_loss: 1.6046 - val_mae: 0.9744
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - loss: 1.5632 - mae: 0.9646 - val_loss: 1.5824 - val_mae: 0.9675
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 28s 977ms/step - loss: 1.3242 - mae: 0.8740 - val_loss: 1.7043 - val_mae: 0.9849
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 41s 980ms/step - loss: 0.9204 - mae: 0.7158 - val_loss: 1.5873 - val_mae: 0.9648
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 27s 937ms/step - loss: 0.3725 - mae: 0.4401 - val_loss: 1.6524 - val_mae: 1.0181
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - loss: 0.1489 - mae: 0.2862 - val_loss: 1.6299 - val_mae: 0.9803
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - loss: 0.0929 - mae: 0.2106 - val_loss: 1.6312 - val_mae: 0.9867
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - loss: 0.06

In [21]:
from tensorflow.keras.layers import Bidirectional

def create_bidirectional_rnn_model(input_shape):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=input_shape))
    model.add(Bidirectional(SimpleRNN(128)))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

bidirectional_rnn_model = create_bidirectional_rnn_model(X_train.shape[1])
bidirectional_rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 103s 3s/step - loss: 10.0366 - mae: 2.5876 - val_loss: 1.7635 - val_mae: 1.0052
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - loss: 1.7222 - mae: 1.0172 - val_loss: 1.6458 - val_mae: 1.0298
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - loss: 1.1465 - mae: 0.8235 - val_loss: 1.9155 - val_mae: 1.0902
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 146s 3s/step - loss: 1.4028 - mae: 0.9244 - val_loss: 1.5475 - val_mae: 0.9726
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 128s 3s/step - loss: 1.2161 - mae: 0.8186 - val_loss: 1.6334 - val_mae: 0.9857
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - loss: 1.2040 - mae: 0.7895 - val_loss: 1.6976 - val_mae: 0.9955
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step - loss: 1.1693 - mae: 0.7223 - val_loss: 1.5429 - val_mae: 0.9572
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step - loss: 0.9465 - mae: 0.5767 - val_loss: 1.5016 - val_mae: 0.9408
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 94s 3s/step - loss: 0.4071 - 

In [22]:
from tensorflow.keras.layers import GRU

def create_gru_model(input_shape):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=input_shape))
    model.add(GRU(128))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

gru_model = create_gru_model(X_train.shape[1])
gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 130s 4s/step - loss: 14.1068 - mae: 3.3977 - val_loss: 2.1512 - val_mae: 1.2120
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 118s 3s/step - loss: 1.8197 - mae: 1.0509 - val_loss: 1.5406 - val_mae: 0.9426
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - loss: 1.1031 - mae: 0.8067 - val_loss: 1.4210 - val_mae: 0.9154
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - loss: 0.5632 - mae: 0.5635 - val_loss: 1.3862 - val_mae: 0.9299
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - loss: 0.2111 - mae: 0.3508 - val_loss: 1.4845 - val_mae: 0.9570
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 91s 3s/step - loss: 0.1245 - mae: 0.2428 - val_loss: 1.5133 - val_mae: 0.9678
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 91s 3s/step - loss: 0.0526 - mae: 0.1548 - val_loss: 1.5972 - val_mae: 1.0108
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step - loss: 0.0824 - mae: 0.1906 - val_loss: 1.6347 - val_mae: 0.9948
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 97s 3s/step - loss: 0.0710 - ma

In [23]:
from tensorflow.keras.layers import LSTM

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=input_shape))
    model.add(LSTM(128))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

lstm_model = create_lstm_model(X_train.shape[1])
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 97s 3s/step - loss: 12.1104 - mae: 2.9542 - val_loss: 1.5627 - val_mae: 0.9810
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - loss: 1.7433 - mae: 1.0305 - val_loss: 1.5449 - val_mae: 0.9763
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - loss: 1.6092 - mae: 0.9784 - val_loss: 1.5758 - val_mae: 0.9585
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 94s 3s/step - loss: 1.3513 - mae: 0.8807 - val_loss: 1.4827 - val_mae: 0.9379
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - loss: 0.8194 - mae: 0.6598 - val_loss: 1.6262 - val_mae: 0.9862
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - loss: 0.3572 - mae: 0.4304 - val_loss: 1.6825 - val_mae: 1.0128
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - loss: 0.2391 - mae: 0.3278 - val_loss: 1.6726 - val_mae: 1.0031
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 97s 3s/step - loss: 0.1491 - mae: 0.2553 - val_loss: 1.7404 - val_mae: 1.0182
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - loss: 0.0770 - mae

In [29]:
# Define the function to evaluate language models
def evaluate_language_models(models, X_test, y_test):
    evaluation_results = {}
    
    for model_name, model in models.items():
        # Evaluate the model using standard metrics
        loss, mae = model.evaluate(X_test, y_test)
        evaluation_results[model_name] = {'Loss': loss, 'MAE': mae}
    
    return evaluation_results

# Define your models and their names
models = {'Simple RNN': rnn_model, 'Bidirectional RNN': bidirectional_rnn_model, 'GRU': gru_model, 'LSTM': lstm_model}

# Evaluate the language models
evaluation_results = evaluate_language_models(models, X_test, y_test)

# Print the evaluation results
for model_name, metrics in evaluation_results.items():
    print(f"{model_name} - Loss: {metrics['Loss']}, MAE: {metrics['MAE']}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step - loss: 1.5414 - mae: 0.9655
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 292ms/step - loss: 1.5577 - mae: 0.9625
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 413ms/step - loss: 1.6102 - mae: 0.9836
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 471ms/step - loss: 1.6052 - mae: 0.9767
Simple RNN - Loss: 1.6302926540374756, MAE: 0.9854066967964172
Bidirectional RNN - Loss: 1.663894534111023, MAE: 0.98110431432724
GRU - Loss: 1.6869354248046875, MAE: 1.0067960023880005
LSTM - Loss: 1.7356089353561401, MAE: 1.009955644607544
